In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
df = pd.read_csv("cleaned_data.csv")
df = df.sample(frac=1)
df['gender'].replace(["Male", "Female"], [0, 1], inplace=True)

/var/folders/f7/j7gv0q6n75n9w5bkr2kqqsz40000gn/T/ipykernel_21431/3680696123.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['gender'].replace(["Male", "Female"], [0, 1], inplace=True)
/var/folders/f7/j7gv0q6n75n9w5bkr2kqqsz40000gn/T/ipykernel_21431/3680696123.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_down

In [3]:
cc_cols = [x for x in df.columns if x[:3] == "cc_"]
med_cols = [x for x in df.columns if x[:5] == "meds_"]
with open("pmh_cols.txt") as f:
    pmh_cols = f.readlines()
pmh_cols = list(map(lambda x:x[:-1], pmh_cols))

x_cols = ["age", "gender"] + cc_cols + pmh_cols
y_cols = ["disposition"] + med_cols

esi_data = pd.get_dummies(df['esi'], dtype=int)

In [4]:
val_split = 0.9

np_x_train = np.array(df[x_cols])[:int(val_split * df.shape[0]), :]
np_esi_train = np.array(esi_data)[:int(val_split * df.shape[0]), :]

np_x_val = np.array(df[x_cols])[int(val_split * df.shape[0]):, :]
np_esi_val = np.array(esi_data)[int(val_split * df.shape[0]):, :]

print(np_x_train.shape)
print(np_esi_train.shape)
print()
print(np_x_val.shape)
print(np_esi_val.shape)

(502216, 483)
(502216, 5)

(55802, 483)
(55802, 5)


In [5]:
gen_model = tf.keras.Sequential([
    tf.keras.layers.Input(483),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(49, activation="softmax")
])

gen_model.compile(
    loss="binary_crossentropy",
    metrics=[
        tf.keras.metrics.AUC(curve="PR"),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall()
    ],
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
)

In [6]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    0.01, decay_steps=5, decay_rate=0.99, staircase=True
)

esi_model = tf.keras.Sequential([
    tf.keras.layers.Input(483),
    tf.keras.layers.Dense(200, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(5, activation="softmax")
])

esi_model.compile(
    loss="categorical_crossentropy",
    metrics=["accuracy"],
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01)
)

In [7]:
esi_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 200)               96800     
                                                                 
 dense_3 (Dense)             (None, 100)               20100     
                                                                 
 dense_4 (Dense)             (None, 5)                 505       
                                                                 
Total params: 117,405
Trainable params: 117,405
Non-trainable params: 0
_________________________________________________________________


In [8]:
esi_model.fit(
    x=np_x_train,
    y=np_esi_train,
    epochs=10,
    validation_data=(np_x_val, np_esi_val),
    batch_size=16,
)

Epoch 1/10
    1/31389 [..............................] - ETA: 1:11:25 - loss: 2.3910 - accuracy: 0.0000e+00

2024-02-09 23:16:44.545102: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


31389/31389 [==============================] - 20s 643us/step - loss: 0.8752 - accuracy: 0.6311 - val_loss: 0.8570 - val_accuracy: 0.6356
Epoch 2/10
31389/31389 [==============================] - 20s 638us/step - loss: 0.8555 - accuracy: 0.6395 - val_loss: 0.8580 - val_accuracy: 0.6471
Epoch 3/10
31389/31389 [==============================] - 20s 640us/step - loss: 0.8533 - accuracy: 0.6406 - val_loss: 0.8421 - val_accuracy: 0.6465
Epoch 4/10
31389/31389 [==============================] - 20s 641us/step - loss: 0.8510 - accuracy: 0.6408 - val_loss: 0.8435 - val_accuracy: 0.6484
Epoch 5/10
31389/31389 [==============================] - 20s 637us/step - loss: 0.8511 - accuracy: 0.6413 - val_loss: 0.8499 - val_accuracy: 0.6461
Epoch 6/10
31389/31389 [==============================] - 20s 638us/step - loss: 0.8511 - accuracy: 0.6417 - val_loss: 0.8503 - val_accuracy: 0.6433
Epoch 7/10
31389/31389 [==============================] - 20s 634us/step - loss: 0.8508 - accuracy: 0.6422 - val_loss